In [1]:
!pip install openai==0.28


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogen 0.3.2 requires openai>=1.3, but you have openai 0.28.0 which is incompatible.
pyautogen 0.3.2 requires openai>=1.3, but you have openai 0.28.0 which is incompatible.


In [2]:
!pip install gensim

In [3]:
!pip install gensim sentence-transformers nltk scikit-learn tqdm


In [4]:
!pip install python-dotenv

In [5]:
import openai
print(openai.__version__)
import json
import os
from dotenv import load_dotenv
import re

# Load environment variables from .env file
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


0.28.0


In [6]:
def load_transcript(file_path):
    """Load the conversation transcript from a text file."""
    with open(file_path, 'r') as file:
        return file.read()


In [7]:
def calculate_duration(transcript_text):
    """Estimate conversation duration based on transcript word count."""
    words = len(transcript_text.split())
    minutes = words // 150  # Average speaking speed is 150 WPM
    seconds = (words % 150) * 60 // 150  # Remaining words converted to seconds
    return f"{minutes:02}:{seconds:02}"


In [8]:
def summarize_transcript(transcript_text):
    """Generate a concise summary using GPT-4o-mini with generalized prompt engineering."""
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  
        messages=[
            {"role": "system", "content": "You are an intelligent summarization assistant for conversations."},
            {
                "role": "user",
                "content": (
                    f"Please summarize the following conversation transcript:\n\n{transcript_text}\n\n"
                    "Provide the summary in JSON format with the following structure:\n"
                    "{\n"
                    "  'summary': 'One or two sentences that provide an overall summary of the conversation.',\n"
                    "  'duration': 'MM:SS',\n"
                    "  'participant_count': [number of unique participants],\n"
                    "  'main_points': [\n"
                    "    '- Describe any roles or responsibilities mentioned by participants.',\n"
                    "    '- Summarize the main topics of discussion in the conversation.',\n"
                    "    '- Note any strong opinions, preferences, or decisions shared by participants.'\n"
                    "  ]\n"
                    "}\n"
                    "Ensure that the main points are flexible and relevant to any type of conversation transcript. "
                    "Make sure to keep the points concise and directly related to the conversation content."
                )
            }
        ],
        max_tokens=300,
        temperature=0.4
    )
    summary_data = response.choices[0].message['content'].strip()
    return summary_data


In [9]:
def process_summary(summary_text, transcript_text):
    """Extract key details from the summary and format them as JSON."""
    
    # Calculate duration dynamically
    duration = calculate_duration(transcript_text)
    
    # Dynamically count unique participants based on conversation markers (e.g., "M:" or "R:" for speakers)
    participant_count = len(set(re.findall(r'^[A-Z]:', transcript_text, re.MULTILINE)))

    # Extract main points in a more general way
    main_points_raw = re.findall(r"- (.+)", summary_text)
    main_points = [re.sub(r'[\\",]+$', '', point.strip()) for point in main_points_raw]  # Clean up any trailing characters

    # Ensure we always have three main points, even if we need to add generic placeholders
    while len(main_points) < 3:
        main_points.append("Point not provided by model.")

    json_output = {
        "summary": "A comprehensive overview of the conversation",
        "duration": duration,
        "participant_count": participant_count,
        "main_points": main_points[:3]  # Limit to 3 main points as requested
    }
    return json_output


In [10]:
def main(file_path):
    # Load transcript
    transcript_text = load_transcript(file_path)
    
    # Get summary from GPT-4o-mini
    summary_text = summarize_transcript(transcript_text)
    
    # Process and format the summary into JSON
    json_output = process_summary(summary_text, transcript_text)
    
    # Print the JSON output
    print(json.dumps(json_output, indent=2))


In [11]:
if __name__ == "__main__":
    main("Round2-Assessment-transcript.txt")


{
  "summary": "A comprehensive overview of the conversation",
  "duration": "61:42",
  "participant_count": 2,
  "main_points": [
    "Renuka is responsible for household decisions, including cooking and health management for her family.",
    "The discussion covers Renuka's daily routine, her family's health concerns, and her approach to purchasing medications.",
    "Renuka prefers to consult doctors for medication and expresses skepticism about generic medicines, emphasizing the importance of doctor recommendations."
  ]
}


In [12]:
!pip install sentence-transformers nltk scikit-learn tqdm


In [13]:

from gensim import corpora
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm, trange

# Load pre-trained Sentence Transformer model for semantic embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and efficient

# Ensure stopwords are downloaded (if not already)
nltk.download('stopwords')
nltk.download('punkt_tab')

2024-11-16 21:08:04.313606: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package stopwords to /Users/naren/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/naren/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [14]:
def preprocess(text, stop_words):
    """
    Tokenizes and preprocesses the input text, removing stopwords and short tokens.
    
    Parameters:
        text (str): The input text to preprocess.
        stop_words (set): A set of stopwords to be removed from the text.
    Returns:
        list: A list of preprocessed tokens.
    """
    result = []
    for token in simple_preprocess(text, deacc=True):
        if token not in stop_words and len(token) > 3:
            result.append(token)
    return result


In [15]:
def get_topic_keywords(transcript, num_topics=10, words_per_topic=5):
    """
    Extracts topics and their associated keywords from the transcript using LDA.

    Parameters:
        transcript (str): The transcript text to process.
        num_topics (int): The number of topics to discover.
        words_per_topic (int): The number of words to include per topic.

    Returns:
        List: A list of dictionaries, each containing a topic name and keywords.
    """
    # Tokenize the transcript into sentences for LDA processing
    sentences = sent_tokenize(transcript)
    
    # Preprocess sentences
    stop_words = set(stopwords.words('english'))
    processed_documents = [preprocess(sentence, stop_words) for sentence in sentences]
    
    # Create a dictionary and corpus
    dictionary = corpora.Dictionary(processed_documents)
    corpus = [dictionary.doc2bow(doc) for doc in processed_documents]
    
    # Build the LDA model
    lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=0)
    
    # Retrieve topics with keywords
    topics = []
    for idx, topic in lda_model.print_topics(num_words=words_per_topic):
        words = [word.split("*")[1].replace('"', '').strip() for word in topic.split(" + ")]
        topics.append({"topic_id": idx, "keywords": words})
    
    return topics


In [16]:
def combine_task_outputs(task1_output, task2_detailed_topics):
    """
    Combines the high-level summary from Task 1 with the detailed topics from Task 2.
    
    Parameters:
        task1_output (dict): The output JSON from Task 1 containing the summary.
        task2_detailed_topics (dict): The detailed topics JSON output from Task 2.
    
    Returns:
        dict: Combined JSON output.
    """
    combined_output = task1_output
    combined_output["detailed_topics"] = task2_detailed_topics["detailed_topics"]
    return combined_output


In [17]:
def generate_topic_summaries(topics, num_topics):
    """
    Generates summaries for each topic using an LLM chat model based on topic keywords.

    Parameters:
        topics (list): A list of topics with associated keywords.
        num_topics (int): The number of topics to summarize.

    Returns:
        list: A list of summaries with detailed descriptions and subtopics.
    """
    prompt_template = '''
For each of the {num_topics} topics, provide a relevant and concise topic description and identify 3 key subtopics. The descriptions and subtopics should be closely connected to the keywords provided.

Format the response like this:

1: Topic Description
- Subtopic 1
- Subtopic 2
- Subtopic 3

Topics: {topics_keywords}
'''

    # Prepare the keywords into a format for the LLM
    topics_keywords = "\n".join([f"{i+1}: {', '.join(topic['keywords'])}" for i, topic in enumerate(topics[:num_topics])])
    prompt = prompt_template.format(num_topics=num_topics, topics_keywords=topics_keywords)
    
    # Call the OpenAI Chat API to generate descriptions
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # Use the correct chat model
        messages=[
            {"role": "system", "content": "You are an intelligent assistant providing topic summaries with subtopics."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400,
        temperature=0.4
    )
    
    # Parse the LLM's response
    topic_summaries = response.choices[0].message['content'].strip().split("\n\n")
    structured_summaries = []
    for summary in topic_summaries:
        lines = summary.strip().split("\n")
        if len(lines) < 4:
            continue
        topic_description = lines[0][3:].strip()  # Remove "1: " prefix
        subtopics = [line[2:].strip() for line in lines[1:4]]  # Remove "- " prefix from subtopics
        structured_summaries.append({
            "topic_name": topic_description,
            "subtopics": subtopics
        })
    
    return structured_summaries


In [18]:
def format_output_for_task2(transcript, topic_summaries, num_topics=10):
    """
    Combines the generated topic summaries with the transcript data in Task 2's required format.

    Parameters:
        transcript (str): The original transcript text.
        topic_summaries (list): A list of summaries generated by the LLM.

    Returns:
        dict: The output in the JSON format specified for Task 2.
    """
    sentences = sent_tokenize(transcript)
    output = []

    # Embed each sentence for better matching
    sentence_embeddings = model.encode(sentences)

    for i, summary in enumerate(topic_summaries[:num_topics]):
        topic_name = summary["topic_name"]
        subtopics = summary["subtopics"]
        
        # Get topic embedding for similarity matching
        topic_embedding = model.encode([topic_name])[0]
        
        # Calculate similarity between topic and each sentence
        similarities = cosine_similarity([topic_embedding], sentence_embeddings).flatten()
        
        # Select top N relevant sentences based on similarity
        top_indices = similarities.argsort()[-5:][::-1]  # Top 5 similar sentences for each topic
        relevant_sentences = [sentences[idx] for idx in top_indices]

        # Prepare structured data for the topic
        topic_data = {
            "topic_name": topic_name,
            "span": f"{top_indices[0]} - {top_indices[-1]}",  # approximate span based on top sentences
            "relevant_excerpts": relevant_sentences,
            "subtopics": subtopics
        }
        output.append(topic_data)
    
    return {"detailed_topics": output}


In [21]:
def task2(file_path, num_topics=7, words_per_topic=5):
    """
    Process the transcript to extract topics, generate summaries, and structure the output.
    """
    with open(file_path, 'r') as file:
        transcript = file.read()
    
    # Step 1: Extract topics using LDA or other topic modeling technique
    topics = get_topic_keywords(transcript, num_topics=num_topics, words_per_topic=words_per_topic)
    
    # Step 2: Generate topic summaries with LLM
    topic_summaries = generate_topic_summaries(topics, num_topics)
    
    # Step 3: Format the output as specified
    detailed_topics = format_output_for_task2(transcript, topic_summaries, num_topics)
    
    # Print the final JSON output
    print(json.dumps(detailed_topics, indent=2))


In [22]:
file_path = "Round2-Assessment-transcript.txt"
task2(file_path)

{
  "detailed_topics": [
    {
      "topic_name": "The Evolution of Tablets Over the Years",
      "span": "520 - 494",
      "relevant_excerpts": [
        "Are you talking about tablets?",
        "I have seen those tablets, but never thought to buy them.",
        "So from there, I got to know that they are government-based tablets.",
        "When it comes to the turn of other tablets like Dolo, crocin etc, when do you get them?",
        "R: We are aware of the name of the tablet and its details, right?"
      ],
      "subtopics": [
        "blets have transformed from simple devices to powerful tools for communication, education, and entertainment. This topic explores their development and impact over time.",
        "Historical milestones in tablet technology",
        "Major brands and their contributions"
      ]
    },
    {
      "topic_name": "Understanding People\u2019s Preferences for Tablets",
      "span": "539 - 544",
      "relevant_excerpts": [
        "But people 